In [1]:
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
import wandb
import multiprocessing


import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
import segmentation_models_pytorch as smp

from models.unet import *
from models.simple_model import *
from models.tri_unet import *
from models.divergent_nets import *
from utils.data_utils.acdc_datamodule import *
from utils.data_utils.data_utils import *
from utils.model_utils.dice_score import *
from utils.model_utils.resnet_loss import ResnetLoss 

from lightning.pytorch.callbacks import RichProgressBar



In [ ]:
pl.seed_everything(42)

In [ ]:
wandb.login()


In [4]:
class SemanticSegmanter(pl.LightningModule):
    def __init__(self, model, learning_rate, criterion) -> None:
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.lr = learning_rate

    def forward(self, x):
        model_output = self.model(x)
        return model_output
    
    def training_step(self, batch, batch_idx):
        images, ground_truths = batch
        masks_pred = self.model(images)
        ground_truths = ground_truths.long()
        loss = self.criterion(masks_pred, ground_truths)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, ground_truths = batch
        masks_pred = self.model(images)
        ground_truths = ground_truths.long()
        loss = self.criterion(masks_pred, ground_truths)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        pred_y = self(x)

        # train dice
        y_pred = torch.argmax(pred_y[2], axis=1)
        y_pred_onehot = F.one_hot(y_pred, 4).permute(0, 3, 1, 2)
        dice = self.compute_dice(y_pred_onehot, y)
        dice_LV = dice[3]
        dice_RV = dice[1]
        dice_MYO = dice[2]
        self.log('dice/all_train_dice', dice[1:].mean(), on_epoch=True, prog_bar=True, logger=True, on_step=False)
        self.log('dice/train_LV_dice', dice_LV, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        self.log('dice/train_RV_dice', dice_RV, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        self.log('dice/train_MYO_dice', dice_MYO, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        # save grad
        for name, params in self.named_parameters():
            if params.grad is not None:
                self.log(f'abs_{name}',params.grad.abs().mean(), on_epoch=True)
        return loss #TODO
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr)

In [5]:
# Constans and Hyperparams
NUM_CLASSES = 4
MAX_EPOCHS = 500

# Big model takes lots of space in memory -> small batch size fits in
BATCH_SIZE_TRAIN = 8
BATCH_SIZE_VAL = 8
BATCH_SIZE_TEST  = 8

In [6]:
transform = DualTransform(20,0.2,0.2)
datamodule = ACDCDataModule("database", BATCH_SIZE_TRAIN,BATCH_SIZE_VAL,BATCH_SIZE_TEST,(256,256,1), convert_to_single=False, transform=transform)
datamodule.setup("fit")

In [7]:
# Diceloss kiegyensúlyozatlan osztályokra
# CrossEntropy is jó lehet, de Dice is jó 
# Jacquard index - IoU
# ignore index kipróbálása -> metrikánál mindenképp érdemes
#  1 ignorált klassznak lehet nincs hatása
#  ignorált osztály -> lehet crossentropy is azonos hatékonyságú
#  osztályonkénti dice coeff
# split betegenként, fixáljuk a validációs halmazt
#  KFold ha nem fixálunk -> ez is lehet ensemble
# pixelszintű tévesztési mátrix, pixel accuracy 
#  legyenek példapredikciók
#  kvalitatív kiértékelés

criterion = smp.losses.DiceLoss(mode="multiclass")
loss = ResnetLoss(criterion)
loss = criterion
# loss = torch.nn.CrossEntropyLoss()

In [8]:
from lightning import Callback

class_labels = {
  0: "background",
  1: "RV",
  2: "myocardium",
  3: "LV"
}

class Visualizer(Callback):
    def on_validation_epoch_start(self, trainer, model):
        x = trainer.datamodule.acdc_val[0][0]
        y = trainer.datamodule.acdc_val[0][1]
        pred = model(x.unsqueeze(0).to('cuda'))
        gt_mask = np.array(y.squeeze())
        pred_mask = np.array(torch.argmax(pred.squeeze().cpu(), dim=0))
        error = (gt_mask == pred_mask).astype(np.uint8)
        trainer.logger.experiment.log(
            {"visualizing":[
                    wandb.Image(x, caption="GT", masks={
                        "segmentation": {
                            "mask_data": gt_mask,
                            "class_labels": class_labels
                        },
                    }),
                    wandb.Image(x, caption="pred", masks={
                        "segmentation": {
                            "mask_data": pred_mask,
                            "class_labels": class_labels
                        },
                    }),
                    wandb.Image(error, caption="error"),
                ]
            })

In [9]:
early_stopping = EarlyStopping(monitor='val_loss',  patience=5 ,mode="min", verbose=True)

In [10]:
from torchvision.models.segmentation import fcn_resnet50

def fcn_facotory():
    model = fcn_resnet50()
    model.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    model.classifier[4] = nn.Conv2d(512, NUM_CLASSES, kernel_size=(1, 1), stride=(1, 1))
    return model

In [12]:
# Unet
unet_model = smp.Unet('resnet34', classes=NUM_CLASSES, in_channels=1)

# Fcn
fcn_model = fcn_facotory()

# TriUnet - 3 unet
feature_models = torch.nn.ModuleList([smp.Unet('resnet34',classes=NUM_CLASSES, in_channels=1) for _ in range(2)])
triunet_3unet_model = TriUnet(feature_models, smp.Unet('resnet34',classes=NUM_CLASSES, in_channels=len(feature_models)*NUM_CLASSES))

# # TriUnet - 3 fcn
# feature_models = torch.nn.ModuleList([fcn_facotory() for _ in range(2)])
# triunet_3fcn_model = TriUnet(feature_models, fcn_facotory())

# TriUnet - 2 fcn + unet
feature_models = torch.nn.ModuleList([fcn_facotory() for _ in range(2)])
triunet_2fcn_1unet_model = TriUnet(feature_models, smp.Unet('resnet34',classes=NUM_CLASSES, in_channels=len(feature_models)*NUM_CLASSES))

models = [
    ("unet_model", unet_model),
    ("fcn_model", fcn_model),
    ("triunet_3unet_model", triunet_3unet_model),
    # ("triunet_3fcn_model", triunet_3fcn_model),
    ("triunet_2fcn_1unet_model", triunet_2fcn_1unet_model)
]

for name, model in models:
    wandb_logger = pl.loggers.WandbLogger(project="Medical Image Segmentation", name=name)
    wandb_logger.watch(model)
    segmenter = SemanticSegmanter(model=model, learning_rate=1e-4, criterion=loss)
    trainer = pl.Trainer(max_epochs=30, logger=wandb_logger, callbacks=[RichProgressBar(), Visualizer(), early_stopping])
    trainer.fit(segmenter, datamodule=datamodule)
    wandb.finish()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\boton/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 70.9MB/s]


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name      ┃ Type     ┃ Params ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model     │ Unet     │ 24.4 M │
│ 1 │ criterion │ DiceLoss │      0 │
└───┴───────────┴──────────┴────────┘

Trainable params: 24.4 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 24.4 M                                                                                               
Total estimated model params size (MB): 97

Output()

...

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name      ┃ Type     ┃ Params ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model     │ FCN      │ 32.9 M │
│ 1 │ criterion │ DiceLoss │      0 │
└───┴───────────┴──────────┴────────┘

Trainable params: 32.9 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 32.9 M                                                                                               
Total estimated model params size (MB): 131

Output()

AttributeError: 'collections.OrderedDict' object has no attribute 'squeeze'

In [8]:
from torchvision.models.segmentation import fcn_resnet50
model1 = smp.Unet('resnet34',classes=NUM_CLASSES, in_channels=1).cuda()
model2 = UNet(n_channels=1, n_classes=4).cuda()
model3 = fcn_resnet50()
model3.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model3.classifier[4] = nn.Conv2d(512, NUM_CLASSES, kernel_size=(1, 1), stride=(1, 1))
model3 = model3.cuda()

In [9]:
import time

criterion = smp.losses.DiceLoss(mode="multiclass")
for ix, m in enumerate([model1, model2, model3]):
    m.train()
    start_time = time.time()
    i = 0
    for x,y in datamodule.train_dataloader():
        out = m(x.to("cuda"))
        if ix == 2:
            loss = ResnetLoss(criterion)
            loss(out, y.to("cuda").long()).backward()
        else:
            criterion(out, y.to("cuda").long()).backward()
        i+=1
        if i>3:
            break
    end_time = time.time()
    
    print(f"Time taken: {end_time - start_time} seconds")

Time taken: 1.7235326766967773 seconds
Time taken: 14.67308259010315 seconds
Time taken: 6.882024526596069 seconds


In [ ]:
@torch.no_grad()
def compute_dice(self, pred_y, y):
        """
        Computes the Dice coefficient for each class in the ACDC dataset.
        Assumes binary masks with shape (num_masks, num_classes, height, width).
        """
        epsilon = 1e-6
        num_masks = pred_y.shape[0]
        num_classes = pred_y.shape[1]
        dice_scores = torch.zeros((num_classes,), device=self.device)

        for c in range(num_classes):
            intersection = torch.sum(pred_y[:, c] * y[:, c])
            sum_masks = torch.sum(pred_y[:, c]) + torch.sum(y[:, c])
            dice_scores[c] = (2. * intersection + epsilon) / (sum_masks + epsilon)
        print(dice_scores)
        return dice_scores

In [ ]:
class Evaluation(Callback):
    def on_test_epoch_start(self, trainer, model):
        x = trainer.datamodule.acdc_val[0][0]
        y = trainer.datamodule.acdc_val[0][1]
        pred = model(x.unsqueeze(0).to('cuda'))
        gt_mask = np.array(y.squeeze())
        pred_mask = np.array(torch.argmax(pred.squeeze().cpu(), dim=0))
        y_pred_onehot = F.one_hot(pred_mask, 4).permute(0, 3, 1, 2)
        dice = compute_dice(y_pred_onehot, gt_mask)
        dice_LV = dice[3]
        dice_RV = dice[1]
        dice_MYO = dice[2]
        self.log('dice/all_train_dice', dice[1:].mean(), on_epoch=True)
        self.log('dice/train_LV_dice', dice_LV, on_epoch=True)
        self.log('dice/train_RV_dice', dice_RV, on_epoch=True)
        self.log('dice/train_MYO_dice', dice_MYO, on_epoch=True)
        # save grad
        for name, params in self.named_parameters():
            if params.grad is not None:
                self.log(f'abs_{name}',params.grad.abs().mean(), on_epoch=True)
        return loss